In [1]:
#https://github.com/m-bain/whisperX/blob/f2da2f858e99e4211fe4f64b5f2938b007827e17/whisperx/diarize.py#L35

In [2]:

%load_ext autoreload
%autoreload 2
import logging
from datetime import datetime, timezone

from app.database_redis.connection import get_redis_client
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings
from app.services.audio.redis import Diarisation,Transcript
import json
import pandas as pd
import numpy as np

logger = logging.getLogger(__name__)

check_and_process_connections_interval_sec: 1.0


/tmp/ipykernel_100058/4266564517.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [4]:
meeting_id = 'gxn-vvzw-uju'
diarization = Diarisation(meeting_id,redis_client=redis_client)
transcript = Transcript(meeting_id,redis_client=redis_client)

In [5]:
meeting = Meeting(redis_client,meeting_id)

In [6]:
def prep_transcripts(new_transcriptions):
    dfs = []
    for t,seek,connection_id in new_transcriptions:
        df = pd.DataFrame(t)[[2,3,4]]
        df.columns = ['start','end','speech']
        df['start'] = pd.to_timedelta(df['start'],unit='s') + pd.Timestamp(seek)
        df['end'] = pd.to_timedelta(df['end'],unit='s') + pd.Timestamp(seek)
        dfs.append(df)
    return pd.concat(dfs)

In [7]:
def prep_diarizations(new_diarizations):
    dfs = []
    for d,t,c in new_diarizations:
        df = pd.DataFrame(d)
        df['start'] = pd.to_timedelta(df['start'])+pd.Timestamp(t)
        df['end']   = pd.to_timedelta(df['end'])+pd.Timestamp(t)
        dfs.append(df)
    return pd.concat(dfs)

In [8]:
final = []

In [9]:
mutable = pd.DataFrame()

In [126]:

# new_diarizations = []
# new_transcriptions = [] 

# final = []

In [130]:

while True:
    d = await diarization.rpop()
    if not d:
        break
    new_diarizations.append(json.loads(d))

while True:
    d = await transcript.rpop()
    if not d:
        break
    new_transcriptions.append(json.loads(d))

In [131]:
len(new_diarizations)

0

In [132]:
len(new_transcriptions)

0

In [109]:
diarize_df = prep_diarizations(new_diarizations)
transcript_segments = prep_transcripts(new_transcriptions)

In [110]:
transcript_segments

,start,end,speech
0,2024-05-24 19:25:07.011206+00:00,2024-05-24 19:25:10.671206+00:00,с простой идеей создать цифровой каталог студ...
1,2024-05-24 19:25:10.711206+00:00,2024-05-24 19:25:14.831206+00:00,"И если вы думаете, что Марк Цукерберг был пер..."
0,2024-05-24 19:25:07.011206+00:00,2024-05-24 19:25:10.031206+00:00,"был первым, кому это пришло в голову, то вы о..."
1,2024-05-24 19:25:10.671206+00:00,2024-05-24 19:25:14.271206+00:00,"Однокурсники Марка, близнецы Кэмерон и Тайлер..."
2,2024-05-24 19:25:14.271206+00:00,2024-05-24 19:25:16.651206+00:00,тоже разрабатывали похожий сайт.
...,...,...,...
0,2024-05-24 19:28:27.180051+00:00,2024-05-24 19:28:34.020051+00:00,"Вообще, если вам попадались ссылки на этот са..."
1,2024-05-24 19:28:34.100051+00:00,2024-05-24 19:28:41.000051+00:00,Яркий и неприятный образ встает из всех этих ...
0,2024-05-24 19:28:27.130051+00:00,2024-05-24 19:28:33.050051+00:00,"этих объявившихся. Да, кстати, говорят два сл..."
1,2024-05-24 19:28:33.050051+00:00,2024-05-24 19:28:38.170051+00:00,частную переписку. Многие добрые люди продолж...


In [111]:
diarize_df

,start,end,speaker,score
0,2024-05-24 19:25:07.011206+00:00,2024-05-24 19:25:07.011206002+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
1,2024-05-24 19:25:07.011206002+00:00,2024-05-24 19:25:07.011206004+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
2,2024-05-24 19:25:07.011206005+00:00,2024-05-24 19:25:07.011206005+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
3,2024-05-24 19:25:07.011206005+00:00,2024-05-24 19:25:07.011206006+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
4,2024-05-24 19:25:07.011206007+00:00,2024-05-24 19:25:07.011206009+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
5,2024-05-24 19:25:07.011206009+00:00,2024-05-24 19:25:07.011206011+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
6,2024-05-24 19:25:07.011206012+00:00,2024-05-24 19:25:07.011206022+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
7,2024-05-24 19:25:07.011206022+00:00,2024-05-24 19:25:07.011206027+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
8,2024-05-24 19:25:07.011206028+00:00,2024-05-24 19:25:07.011206032+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265
9,2024-05-24 19:25:07.011206032+00:00,2024-05-24 19:25:07.011206034+00:00,01fc2af8-bf57-4225-a682-711192e2e3ff,0.265265


In [782]:

    
    
diarize_df = prep_diarizations(new_diarizations)
transcript_segments = prep_transcripts(new_transcriptions).to_dict('records')
for seg in transcript_segments:

    diarize_df['intersection'] = np.minimum(diarize_df['end'], seg['end']) - np.maximum(diarize_df['start'], seg['start'])
    # remove no hit, otherwise we look for closest (even negative intersection...)
    dia_tmp = diarize_df[diarize_df['intersection']<pd.Timedelta(0)]
    if len(dia_tmp) > 0:
        # sum over speakers
        speaker = dia_tmp.groupby("speaker")["intersection"].sum().sort_values(ascending=False).index[0]
        seg["speaker"] = speaker
        


,speech,start,end,speaker
switched,,,,
7,"Те, кто придут, в общем, присоединятся к прямому эфиру программы.",2024-05-24 18:42:51.907225+00:00,2024-05-24 18:42:57.047225+00:00,pasha
8,"Но перед Лимассолом мы еще зайдем в Стокгольм, и там мы будем.",2024-05-24 18:42:57.047225+00:00,2024-05-24 18:42:59.847225+00:00,923f3fed-c6e6-45f8-a3f1-80d53950511e
9,"Те, кто придут, в общем, присоединятся к прямому эфиру программы. Но перед Лимасовым мы еще зайдем в Стокгольм, там мы будем говорить о конституционных проектах, проектах декабристов. Ну а теперь начинаем. Не новости, но события.",2024-05-24 18:43:15.226410+00:00,2024-05-24 18:43:32.346410+00:00,katya
10,Но о событиях. С какого события мы начнем?,2024-05-24 18:43:15.916410+00:00,2024-05-24 18:43:19.216410+00:00,923f3fed-c6e6-45f8-a3f1-80d53950511e
11,"Как обычно, с того, о котором мы говорили и в прошлом, и даже, можно сказать, в прошлом.",2024-05-24 18:43:19.636410+00:00,2024-05-24 18:43:23.336410+00:00,katya
12,"...и в прошлом, и даже, можно сказать, в позапрошлом нашем выпуске.",2024-05-24 18:43:15.226410+00:00,2024-05-24 18:43:18.686410+00:00,923f3fed-c6e6-45f8-a3f1-80d53950511e
13,"Итак, напомню, дорогим слушателям, что у нас сформирован полностью кабинет министров...",2024-05-24 18:43:18.686410+00:00,2024-05-24 18:43:23.046410+00:00,katya
14,Сформирован полностью кабинет министров в гражданской своей и в силовой части.,2024-05-24 18:43:15.226410+00:00,2024-05-24 18:43:19.466410+00:00,923f3fed-c6e6-45f8-a3f1-80d53950511e
15,"Также произошли некоторые перерассадки в администрации президента. Также сейчас поговорим о том, как... Также сейчас поговорим о том, как меняется власть в некоторых регионах. Что успел произойти с прошлого выпуска до... Прошлого выпуска до нынешнего. Представитель правительства подписал распоряжение о распределении обязанностей между своими заместителями, между первыми и простыми вице-премьерами. Напомню, что одна из немногих структурных новаций, которые мы увидели в этом новом, старом, на самом деле довольно консервативном по составу правительстве. Консерватизм по составу правительства – это развод между вице-премьерами и министрами. То есть теперь все вице-премьеры, кроме трудного представителя президента в федеральном округе и того вице-премьера,",2024-05-24 18:43:19.746410+00:00,2024-05-24 18:43:52.294967+00:00,katya


In [783]:
# assign speaker to words
if 'words' in seg:
    for word in seg['words']:
        if 'start' in word:
            diarize_df['intersection'] = np.minimum(diarize_df['end'], word['end']) - np.maximum(diarize_df['start'], word['start'])
            diarize_df['union'] = np.maximum(diarize_df['end'], word['end']) - np.minimum(diarize_df['start'], word['start'])
            # remove no hit
            if not fill_nearest:
                dia_tmp = diarize_df[diarize_df['intersection'] > 0]
            else:
                dia_tmp = diarize_df
            if len(dia_tmp) > 0:
                # sum over speakers
                speaker = dia_tmp.groupby("speaker")["intersection"].sum().sort_values(ascending=False).index[0]
                word["speaker"] = speaker


In [785]:
transcript_segments

[{'start': Timestamp('2024-05-24 18:42:20.616054+0000', tz='UTC'),
  'end': Timestamp('2024-05-24 18:42:30.866054+0000', tz='UTC'),
  'speech': ' В эфире программа Екатерины Шульман «Статус».',
  'speaker': 'ca1aad5e-56c3-4ff3-9621-76fcbd678084'},
 {'start': Timestamp('2024-05-24 18:42:20.736054+0000', tz='UTC'),
  'end': Timestamp('2024-05-24 18:42:21.736054+0000', tz='UTC'),
  'speech': ' Статус.',
  'speaker': 'ca1aad5e-56c3-4ff3-9621-76fcbd678084'},
 {'start': Timestamp('2024-05-24 18:42:51.907225+0000', tz='UTC'),
  'end': Timestamp('2024-05-24 18:42:55.147225+0000', tz='UTC'),
  'speech': ' выходит сразу на трех youtube каналах на канале екатерины шульман',
  'speaker': 'ca1aad5e-56c3-4ff3-9621-76fcbd678084'},
 {'start': Timestamp('2024-05-24 18:42:55.147225+0000', tz='UTC'),
  'end': Timestamp('2024-05-24 18:42:59.707225+0000', tz='UTC'),
  'speech': ' здравствуйте екатерина михайловна на канале на русском и на канале живой',
  'speaker': '923f3fed-c6e6-45f8-a3f1-80d53950511e'},

In [784]:
pd.DataFrame(transcript_segments)

,start,end,speech,speaker
0,2024-05-24 18:42:20.616054+00:00,2024-05-24 18:42:30.866054+00:00,В эфире программа Екатерины Шульман «Статус».,ca1aad5e-56c3-4ff3-9621-76fcbd678084
1,2024-05-24 18:42:20.736054+00:00,2024-05-24 18:42:21.736054+00:00,Статус.,ca1aad5e-56c3-4ff3-9621-76fcbd678084
2,2024-05-24 18:42:51.907225+00:00,2024-05-24 18:42:55.147225+00:00,выходит сразу на трех youtube каналах на канале екатерины шульман,ca1aad5e-56c3-4ff3-9621-76fcbd678084
3,2024-05-24 18:42:55.147225+00:00,2024-05-24 18:42:59.707225+00:00,здравствуйте екатерина михайловна на канале на русском и на канале живой,923f3fed-c6e6-45f8-a3f1-80d53950511e
4,2024-05-24 18:42:59.707225+00:00,2024-05-24 18:43:02.967225+00:00,гость также конечно приложение эхо если еще не,923f3fed-c6e6-45f8-a3f1-80d53950511e
5,2024-05-24 18:42:51.907225+00:00,2024-05-24 18:42:59.767225+00:00,в приложении эхо если еще не скачали то обязательно скачайте веб-стор или google play я скажу что сегодня будет,ca1aad5e-56c3-4ff3-9621-76fcbd678084
6,2024-05-24 18:42:52.147225+00:00,2024-05-24 18:42:53.847225+00:00,"скажу, что сегодня будет",923f3fed-c6e6-45f8-a3f1-80d53950511e
7,2024-05-24 18:42:53.847225+00:00,2024-05-24 18:42:54.467225+00:00,сразу,923f3fed-c6e6-45f8-a3f1-80d53950511e
8,2024-05-24 18:42:54.467225+00:00,2024-05-24 18:42:56.827225+00:00,все рубрики.,923f3fed-c6e6-45f8-a3f1-80d53950511e
9,2024-05-24 18:42:57.967225+00:00,2024-05-24 18:42:58.747225+00:00,Надеемся на это.,923f3fed-c6e6-45f8-a3f1-80d53950511e


In [ ]:
pd.set_option('display.max_colwidth', None)
final.append(pd.DataFrame(transcript_segments))
concatinated = pd.concat(final)
concatinated['switched'] = concatinated['speaker']!=concatinated['speaker'].shift()
concatinated['switched'] = concatinated['switched'].cumsum()
concatinated = concatinated.groupby('switched').agg({"speech":'sum','start':'first','end':'last','speaker':'first'})


concatinated = concatinated.replace({'ca1aad5e-56c3-4ff3-9621-76fcbd678084':'pasha',
                                     'f0097512-c612-48f1-8348-f7a0c6145e25':'katya'})

concatinated.tail(10)